# Measuring Quantum Volume

![hero:quantum volume](images/quantum_volume.png)


## Why quantum volume?

Comparing different quantum computers is not an easy task. There are many different factors that affect the performance of a quantum computer and it’s not always clear which factors will have the biggest effect when doing useful quantum computations.

Say you were designing your quantum computer: Would you trade higher fidelity gates (with less errors) for reduced qubit connectivity (requiring longer circuits)? Maybe you have a great [transpiler](gloss:transpiler) that can organise the swaps so reduced connectivity isn’t as much of an issue, or maybe the readout errors are so high that any improvement in gate fidelity is negligible anyway? We can add each of these statistics to our quantum computer scorecard, but the exact importance of each element is up for debate. Additionally, even if we scored highly at each individual metric, there could still be other unforeseen problems that arise when running quantum circuits.

One thing is certain, we need many qubits and the ability to manipulate them precisely in order to achieve a [quantum advantage](gloss:quantumadvantage). If we do not have enough qubits, we cannot do any useful computations, and if these qubits are not precise or reliable enough, then our measurements will be meaningless.


::: .tabset

## What affects computing power?


::: tab

### Qubit count

**Number of qubits (more is better)**

The number of qubits in our quantum computer sets an upper limit on the power of the computations it can do. To gain advantage over classical computers, we want to be able to entangle as many qubits as possible.

![](images/qubit_count.png)

::: tab

### Readout errors

**Gate and readout errors (less is better)**

As mentioned above, the errors in our quantum computer also set an upper limit on the power of our quantum computer. If we can’t reliably manipulate our quantum states, then we’re not going to get meaningful results from our quantum computer.

    figure: x-img(src="images/readout_errors.png" width=194 height=197)

::: tab

### Connectivity

**Qubit-Qubit connectivity (more is better)**

If two qubits cannot communicate directly, we need to add extra ‘swap’ gates to move them to the right place, increasing the length of our computation. Quantum volume circuits assume full connectivity, so if you need to insert extra swap gates, that’s on you!

    figure: x-img(src="images/connectivity.png" width=199 height=189)

::: tab

### Gate set

**Gate set (larger / more powerful is better)**

Quantum volume asks for random two-qubit gates, but working out how to implement them is also on you. If your hardware supports more gates, you’re more likely to get decent results.

    figure: x-img(src="images/gate_set.png" width=302 height=140)

::: tab

### Software stack

**Compilers and software stack (more intelligent is better)**

Some of the other problems (inserting extra swap gates, creating the specified gates from your available gate set, and dealing with noise) can be mitigated using intelligent classical software such as the transpiler.

    figure: x-img(src="images/software_stack.png" width=308 height=150)

:::

:::


This is where quantum volume comes in. Quantum volume is a holistic benchmark, instead of looking at the details, we simply measure how good the quantum computer is at doing the thing we want it to do: Executing quantum circuits. The quantum volume test creates randomized circuits to a specification, and the ‘score’ of the quantum computer increases with the size of the circuit it can reliably execute.

In this chapter, we will learn:

1.	What quantum volume is.
2.	How to create a random square circuit.
3.	How to see if a device can achieve a certain quantum volume.
4.	How to use Qiskit’s tools to make this easier for us.


## What is Quantum Volume?

Quantum volume (QV) is a single-number metric used to measure the power of a quantum computer. It’s used for near-term devices with a modest number of qubits, and measures the largest random circuit of equal depth and width that can be reliably executed.


::: .tabset.demo

## Visual demo

::: tab

![vue:what-is-quantum-volume](images/quantum_volume_2.png)

:::

:::
